In [43]:
import pandas as pd
import numpy as np

In [44]:
#read data
calories14 = pd.read_json('../osfstorage-archive/pmdata/p14/fitbit/calories.json')
distance14 = pd.read_json('../osfstorage-archive/pmdata/p14/fitbit/distance.json')
l_act14 = pd.read_json('../osfstorage-archive/pmdata/p14/fitbit/lightly_active_minutes.json')
m_act14 = pd.read_json('../osfstorage-archive/pmdata/p14/fitbit/moderately_active_minutes.json')
r_heart14 = pd.read_json('../osfstorage-archive/pmdata/p14/fitbit/resting_heart_rate.json')
sedentary14 = pd.read_json('../osfstorage-archive/pmdata/p14/fitbit/sedentary_minutes.json')
sleep_score14 = pd.read_csv('../osfstorage-archive/pmdata/p14/fitbit/sleep_score.csv')
sleep14 = pd.read_json('../osfstorage-archive/pmdata/p14/fitbit/sleep.json')
steps14 = pd.read_json('../osfstorage-archive/pmdata/p14/fitbit/steps.json')
hr_zone14 = pd.read_json('../osfstorage-archive/pmdata/p14/fitbit/time_in_heart_rate_zones.json')
v_act14 = pd.read_json('../osfstorage-archive/pmdata/p14//fitbit/very_active_minutes.json')



In [45]:
#data frequency to date using sum
calories_processed = calories14.resample('D', on = 'dateTime').value.sum()
distance_processed = distance14.resample('D', on = 'dateTime').value.sum()
steps_processed = steps14.resample('D', on = 'dateTime').value.sum()

In [46]:
#resting heart rate to simple df
r_heart14['value']=r_heart14['value'].apply(lambda x: x['value'])
r_heart_processed=r_heart14

In [47]:
#delete time in sleep score time stamp
sleep_score14['timestamp']=sleep_score14['timestamp'].apply(lambda x: x[:10])
#drop unnecessary column
sleep_score_processed = sleep_score14.drop(columns='sleep_log_entry_id') #drop entry id

In [48]:
#dictionary with heart rate zone minutes to df
hr_zone_value = pd.DataFrame([eval(str(a).replace('{\'valuesInZones\': ','').replace('}}','}')) for a in hr_zone14['value']])
hr_zone_processed = pd.concat([hr_zone14['dateTime'], hr_zone_value], axis=1)

In [49]:
#unify index
l_act_processed = l_act14.set_index('dateTime')
m_act_processed = m_act14.set_index('dateTime')
r_heart_processed = r_heart_processed.set_index('dateTime')
sedentary_processed = sedentary14.set_index('dateTime')
sleep_score_processed = sleep_score_processed.set_index('timestamp')
sleep_processed = sleep14.set_index('dateOfSleep')
hr_zone_processed = hr_zone_processed.set_index('dateTime')
v_act_processed = v_act14.set_index('dateTime')
#index to DateTimeIndex
sleep_score_processed.index = pd.to_datetime(sleep_score_processed.index)

In [50]:
#get dummy values for mainsleep true(1)/false(0)
sleep_processed['mainSleep'] = pd.get_dummies(sleep_processed['mainSleep'])[True].fillna(0)
#drop unnecessary columns
sleep_processed = sleep_processed.drop(['logId','startTime','endTime','type','infoCode','levels'], axis=1)
#index to DateTimeIndex
sleep_processed.index = pd.to_datetime(sleep_processed.index)

In [57]:
#read prediction target
wellness = pd.read_csv('../osfstorage-archive/pmdata/p14/pmsys/wellness.csv') 
wellness['effective_time_frame'] = wellness['effective_time_frame'].apply(lambda x: x[:10])
wellness = wellness.rename(columns={"effective_time_frame": "dateTime"})
readiness = wellness.loc[:,['dateTime','readiness']].set_index('dateTime')
readiness.index = pd.to_datetime(readiness.index)

In [58]:
#drop duplicated records
readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed = [x[~x.index.duplicated(keep='first')] for x in [readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed]]

In [59]:
#concat data
P14 = pd.concat([readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed], axis=1)

In [54]:
P14
# i do not know why there are 31 columns?

,value,value,value,value,value,value,overall_score,composition_score,revitalization_score,duration_score,...,mainSleep,value,IN_DEFAULT_ZONE_2,BELOW_DEFAULT_ZONE_1,IN_DEFAULT_ZONE_1,IN_DEFAULT_ZONE_3,BELOW_CUSTOM_ZONE,IN_CUSTOM_ZONE,ABOVE_CUSTOM_ZONE,value
2019-11-13,1627.20,NaN,0.0,0.0,0.000000,1440.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2019-11-14,2373.37,680250.0,133.0,23.0,62.500000,1266.0,NaN,NaN,NaN,NaN,...,NaN,8725.0,1.0,906.0,95.0,0.0,NaN,NaN,NaN,18.0
2019-11-15,3261.86,1224530.0,234.0,37.0,66.217685,629.0,69.0,18.0,16.0,35.0,...,1.0,16612.0,20.0,1116.0,217.0,21.0,NaN,NaN,NaN,77.0
2019-11-16,2229.42,450430.0,173.0,13.0,66.184764,373.0,75.0,18.0,16.0,41.0,...,1.0,6010.0,3.0,1241.0,59.0,1.0,NaN,NaN,NaN,10.0
2019-11-17,2392.07,683050.0,168.0,9.0,64.533855,660.0,83.0,18.0,20.0,45.0,...,1.0,8845.0,0.0,1341.0,39.0,0.0,13.0,1367.0,0.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-28,NaN,280840.0,52.0,12.0,61.707156,945.0,75.0,20.0,18.0,37.0,...,1.0,3744.0,0.0,704.0,14.0,0.0,1.0,717.0,0.0,12.0
2020-03-29,NaN,2567170.0,154.0,22.0,61.742184,601.0,71.0,21.0,16.0,34.0,...,1.0,27173.0,99.0,779.0,61.0,0.0,28.0,911.0,0.0,150.0
2020-03-30,NaN,122820.0,83.0,0.0,61.638319,1024.0,68.0,21.0,15.0,32.0,...,1.0,1640.0,0.0,1003.0,0.0,0.0,1.0,1002.0,0.0,0.0
2020-03-31,NaN,768830.0,149.0,3.0,60.244677,835.0,NaN,NaN,NaN,NaN,...,1.0,8586.0,24.0,1155.0,35.0,0.0,38.0,1176.0,0.0,37.0


In [69]:
#drop unnecessary column
P14 = P14.drop(columns='IN_CUSTOM_ZONE') #drop entry id

In [70]:
P14 = P14.drop(columns='ABOVE_CUSTOM_ZONE') #drop entry id

In [72]:
P14 = P14.drop(columns='IN_DEFAULT_ZONE_2') #drop entry id


In [73]:
P14.columns=['calories','distance','l_act','m_act','resting_heart','sedentary','sleep_overall_score','sleep_composition_score',
             'sleep_revitalization_score', 'sleep_duration_score',
             'deep_sleep_in_minutes', 'sleep_resting_heart_rate','sleep_restlessness',
             'sleep_duration','sleep_minutesToFallAsleep','sleep_minutesAsleep','sleep_minutesAwake',
             'sleep_minutesAfterWakeup','sleep_timeInBed','sleep_efficiency',
             'main_sleep','steps',
             'hr_below_zone1', 'hr_zone1', 'hr_zone3','hr_zone2','v_act']

In [74]:
P14

,calories,distance,l_act,m_act,resting_heart,sedentary,sleep_overall_score,sleep_composition_score,sleep_revitalization_score,sleep_duration_score,...,sleep_minutesAfterWakeup,sleep_timeInBed,sleep_efficiency,main_sleep,steps,hr_below_zone1,hr_zone1,hr_zone3,hr_zone2,v_act
2019-11-13,NaN,1627.20,NaN,0.0,0.0,0.000000,1440.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2019-11-14,NaN,2373.37,680250.0,133.0,23.0,62.500000,1266.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8725.0,906.0,95.0,0.0,18.0
2019-11-15,NaN,3261.86,1224530.0,234.0,37.0,66.217685,629.0,69.0,18.0,16.0,...,70.0,0.0,463.0,94.0,1.0,16612.0,1116.0,217.0,21.0,77.0
2019-11-16,NaN,2229.42,450430.0,173.0,13.0,66.184764,373.0,75.0,18.0,16.0,...,141.0,0.0,726.0,92.0,1.0,6010.0,1241.0,59.0,1.0,10.0
2019-11-17,5.0,2392.07,683050.0,168.0,9.0,64.533855,660.0,83.0,18.0,20.0,...,78.0,0.0,564.0,91.0,1.0,8845.0,1341.0,39.0,0.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-28,NaN,NaN,280840.0,52.0,12.0,61.707156,945.0,75.0,20.0,18.0,...,42.0,2.0,419.0,98.0,1.0,3744.0,704.0,14.0,0.0,12.0
2020-03-29,6.0,NaN,2567170.0,154.0,22.0,61.742184,601.0,71.0,21.0,16.0,...,95.0,0.0,573.0,0.0,1.0,27173.0,779.0,61.0,0.0,150.0
2020-03-30,5.0,NaN,122820.0,83.0,0.0,61.638319,1024.0,68.0,21.0,15.0,...,44.0,0.0,333.0,95.0,1.0,1640.0,1003.0,0.0,0.0,0.0
2020-03-31,NaN,NaN,768830.0,149.0,3.0,60.244677,835.0,NaN,NaN,NaN,...,61.0,0.0,416.0,98.0,1.0,8586.0,1155.0,35.0,0.0,37.0
